## Generating BJT using pure GDSfactory

In [1]:
from glayout import MappedPDK, sky130 , gf180
#from gdsfactory.cell import cell
from gdsfactory import Component
from gdsfactory.geometry import boolean
from gdsfactory.components import text_freetype, rectangle, array, rectangular_ring

In [2]:
top_level=Component(name="BJT")
pdk=gf180
pdk.activate()

2025-09-02 07:24:35.699 | INFO     | gdsfactory.pdk:activate:337 - 'gf180' PDK is now active


In [3]:
pdk.__dict__

{'name': 'gf180',
 'cross_sections': {},
 'cells': {},
 'symbols': {},
 'default_symbol_factory': functools.partial(<function floorplan_with_block_letters at 0x79b45de04940>, prefix='SYMBOL_floorplan_with_block_letters'),
 'base_pdk': None,
 'default_decorator': None,
 'layers': {'metal5': (81, 0),
  'via4': (41, 0),
  'metal4': (46, 0),
  'via3': (40, 0),
  'metal3': (42, 0),
  'via2': (38, 0),
  'metal2': (36, 0),
  'via1': (35, 0),
  'metal1': (34, 0),
  'contact': (33, 0),
  'poly2': (30, 0),
  'comp': (22, 0),
  'nplus': (32, 0),
  'pplus': (31, 0),
  'nwell': (21, 0),
  'lvpwell': (204, 0),
  'dnwell': (12, 0),
  'CAP_MK': (117, 5),
  'drc_bjt': (127, 5),
  'lvs_bjt': (118, 5),
  'metal5_label': (81, 10),
  'metal4_label': (46, 10),
  'metal3_label': (42, 10),
  'metal2_label': (36, 10),
  'metal1_label': (34, 10),
  'poly2_label': (30, 10),
  'comp_label': (22, 10)},
 'layer_stack': None,
 'layer_views': None,
 'layer_transitions': {},
 'sparameters_path': None,
 'modes_path': N

In [4]:
cdims=(8.28, 8.28)
bdims=(6.8,6.8)
edims=(5.32,5.32)
bjt_layers=("p+s/d","n+s/d","p+s/d")
bjt_dims=(edims,bdims,cdims)

In [5]:
e_diff= rectangle(size=bjt_dims[0],layer=pdk.get_glayer(bjt_layers[0]),centered=True)
bt_diff= rectangle(size=bjt_dims[1],layer=pdk.get_glayer(bjt_layers[1]),centered=True)
ct_diff= rectangle(size=bjt_dims[2],layer=pdk.get_glayer(bjt_layers[2]),centered=True)
b_diff= boolean(A=bt_diff, B=e_diff, operation="not", 
                layer=pdk.get_glayer(bjt_layers[1]))
c_diff= boolean(A=ct_diff, B=bt_diff, operation="not",
                layer=pdk.get_glayer(bjt_layers[2]))
top_level.add_ref(e_diff)
top_level.add_ref(b_diff)
top_level.add_ref(c_diff)

ComponentReference (parent Component "boolean_a37f36cf", ports [], origin (0.0, 0.0), rotation 0.0, x_reflection False)

In [6]:
## Adding nwell
nw = rectangle(size=bjt_dims[1],layer=pdk.get_glayer("nwell"),centered=True)
top_level.add_ref(nw)

ComponentReference (parent Component "rectangle_854c7477", ports ['e1', 'e2', 'e3', 'e4'], origin (0.0, 0.0), rotation 0.0, x_reflection False)

In [7]:
comp_min_sep=0.16
contact_dims=(0.22, 0.22)
contact_inner_sep=0.25
contact_comp_sep=0.1

In [8]:
import numpy as np

In [9]:
## Adding comp
comp_layer_name="active_diff"
comp_e= rectangle(size=tuple(np.asarray(bjt_dims[0])-2*comp_min_sep),
                  layer=pdk.get_glayer(comp_layer_name),centered=True)
comp_out_b= rectangle(size=tuple(np.asarray(bjt_dims[1])-2*comp_min_sep),
                      layer=pdk.get_glayer(comp_layer_name),centered=True)
comp_inn_b= rectangle(size=tuple(np.asarray(bjt_dims[0])+2*comp_min_sep),
                      layer=pdk.get_glayer(comp_layer_name),centered=True)
comp_out_c= rectangle(size=tuple(np.asarray(bjt_dims[2])-2*comp_min_sep),
                      layer=pdk.get_glayer(comp_layer_name),centered=True)
comp_inn_c= rectangle(size=tuple(np.asarray(bjt_dims[1])+2*comp_min_sep),
                      layer=pdk.get_glayer(comp_layer_name),centered=True)

comp_b= boolean(A=comp_out_b, B=comp_inn_b, operation="not", 
                layer=pdk.get_glayer(comp_layer_name))
comp_c= boolean(A=comp_out_c, B=comp_inn_c, operation="not",
                layer=pdk.get_glayer(comp_layer_name))
top_level.add_ref(comp_e)
top_level.add_ref(comp_b)
top_level.add_ref(comp_c)

ComponentReference (parent Component "boolean_52fbfc61", ports [], origin (0.0, 0.0), rotation 0.0, x_reflection False)

In [10]:
#top_level.show()

In [11]:
# Adding contacts
comp_min_sep=0.16
contact_dims=(0.22, 0.22)
contact_inner_sep=0.25
contact_array_sep=0.28
contact_comp_sep=0.1

contact_layer_name="mcon"
contact= rectangle(size=contact_dims,
                  layer=pdk.get_glayer(contact_layer_name),centered=True)

n_contacts_e=np.floor((np.asarray(edims)-
              2*comp_min_sep-
              2*contact_comp_sep-
              np.asarray(contact_dims)
             )/(np.asarray(contact_dims)+contact_array_sep)+
                      np.asarray((1,1)))

In [12]:
n_contacts_b=np.floor((np.asarray(bdims)-
              2*comp_min_sep-
              2*contact_comp_sep-
              np.asarray(contact_dims)-
              np.asarray((1,0))*(np.asarray(bdims)-np.asarray(edims)-2*comp_min_sep)
             )/(np.asarray(contact_dims)+contact_inner_sep)+
                      np.asarray((1,1)))

n_contacts_c=np.floor((np.asarray(cdims)-
              2*comp_min_sep-
              2*contact_comp_sep-
              np.asarray(contact_dims)-
              np.asarray((1,0))*(np.asarray(cdims)-np.asarray(bdims)-2*comp_min_sep)
             )/(np.asarray(contact_dims)+contact_inner_sep)+
                      np.asarray((1,1)))

In [13]:
from decimal import Decimal, ROUND_HALF_UP
def round(x):
    return float(Decimal(x).quantize(Decimal("0.001"), 
                                 rounding=ROUND_HALF_UP)) 

def get_half_contact_array_size_m1(n_contacts,contact_inner_sep,contact_dims):
    return (n_contacts-1)*(contact_inner_sep +np.asarray(contact_dims))/2

def get_mid_ring(adims, bdims):
    return (np.asarray(adims)+np.asarray(bdims))/2 

def get_shift(position, n_contacts,contact_inner_sep, contact_dims, mid_ring):
    half_array_contact_size =  get_half_contact_array_size_m1(n_contacts,contact_inner_sep,contact_dims)
    
    if position=="top":
        hshift=(-1,0)*half_array_contact_size
        vshift=(0,1)*mid_ring/2
    elif position=="left":
        hshift=(-1,0)*mid_ring/2
        vshift=(0,-1)*half_array_contact_size
    elif position=="right":
        hshift=(1, 0)*mid_ring/2
        vshift=(0,-1)*half_array_contact_size
    elif position=="bottom":
        hshift=(-1, 0)*half_array_contact_size
        vshift=(0,-1)*mid_ring/2
    else:
        raise ValueError(f"Not a valid position: {position}")
    return hshift+vshift


In [14]:
mid_ring_b_e=get_mid_ring(edims, bdims)
mid_ring_c_b=get_mid_ring(bdims, cdims)
get_shift("bottom", n_contacts_b,contact_inner_sep, contact_dims, mid_ring_b_e)

array([-2.35, -3.03])

In [15]:
def add_ring_contacts(parent, reference,n_contacts, mid_ring, contact_inner_sep,contact_dims):
    positions=["top", "bottom", "left", "right"]
    columns=[int(n_contacts[0]),int(n_contacts[0]),1,1]
    rows=[1, 1, int(n_contacts[1]),int(n_contacts[1])]
    
    refs=[]
    for position,n_column,n_row in list(zip(positions,columns,rows)):
    
        cref=parent.add_ref(reference, 
                         columns=n_column, 
                         rows=n_row, 
                         spacing=tuple(contact_inner_sep +np.asarray(contact_dims))
                        )
        cref.move(tuple([round(x)
               for x in get_shift(position, n_contacts,contact_inner_sep, contact_dims, mid_ring)]))
        refs.append(cref)
    return refs

def add_fill_contacts(parent, reference,n_contacts, contact_array_sep,contact_dims):

    ref=parent.add_ref(reference, 
                     columns=int(n_contacts[0]), 
                     rows=int(n_contacts[1]), 
                     spacing=tuple(contact_array_sep +np.asarray(contact_dims))
                    )
    ref.move(tuple(np.asarray((-1,-1))*(n_contacts-1)*(contact_array_sep +np.asarray(contact_dims))/2))
    return ref

In [16]:
ccontacts=Component()
refs_e=add_fill_contacts(ccontacts,contact,n_contacts_e,contact_array_sep,contact_dims)
refs_b=add_ring_contacts(ccontacts,contact,n_contacts_b,mid_ring_b_e,contact_inner_sep,contact_dims)
refs_c=add_ring_contacts(ccontacts,contact,n_contacts_c,mid_ring_c_b,contact_inner_sep,contact_dims)


In [17]:
ccontacts.show()

/headless/conda-env/miniconda3/envs/GLdev/lib/python3.10/site-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 1 in 'Unnamed_2b50e34e'
  gdspath = component.write_gds(
2025-09-02 07:24:39.423 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.3", "klayout_version": "0.30.2", "type": "open", "file": "/tmp/gdsfactory/Unnamed_2b50e34e.gds"}


In [18]:
top_level.add_ref(ccontacts)

ComponentReference (parent Component "Unnamed_2b50e34e", ports [], origin (0.0, 0.0), rotation 0.0, x_reflection False)

In [19]:
top_level.show()

/headless/conda-env/miniconda3/envs/GLdev/lib/python3.10/site-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 1 in 'BJT'
  gdspath = component.write_gds(
2025-09-02 07:24:39.967 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.3", "klayout_version": "0.30.2", "type": "open", "file": "/tmp/gdsfactory/BJT.gds"}


In [20]:
import gdsfactory as gf

In [21]:
rd=gf.components.rectangular_ring(enclosed_size=edims,layer=pdk.get_glayer(contact_layer_name), width=0.74, centered=True)

In [22]:
## Adding metal connection
metal_layer_name="met1"
metal_contacts=Component()
m1_e= rectangle(size=tuple(np.asarray(bjt_dims[0])-2*comp_min_sep),
                  layer=pdk.get_glayer(metal_layer_name),centered=True)

m1_b=rectangular_ring(enclosed_size=tuple(np.asarray(bjt_dims[0])+2*comp_min_sep),
                      layer=pdk.get_glayer(metal_layer_name), 
                      width=round((np.asarray(bjt_dims[1][0])-np.asarray(bjt_dims[0][0])-4*comp_min_sep)/2), 
                      centered=True)
m1_c=rectangular_ring(enclosed_size=tuple(np.asarray(bjt_dims[1])+2*comp_min_sep),
                      layer=pdk.get_glayer(metal_layer_name), 
                      width=round((np.asarray(bjt_dims[2][0])-np.asarray(bjt_dims[1][0])-4*comp_min_sep)/2), 
                      centered=True)
metal_contacts.add_ref(m1_e)
metal_contacts.add_ref(m1_b)
metal_contacts.add_ref(m1_c)

ComponentReference (parent Component "rectangular_ring_7e769340", ports [], origin (0.0, 0.0), rotation 0.0, x_reflection False)

In [23]:
metal_contacts.show()

/headless/conda-env/miniconda3/envs/GLdev/lib/python3.10/site-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 1 in 'Unnamed_050aa4c5'
  gdspath = component.write_gds(
2025-09-02 07:24:40.992 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.3", "klayout_version": "0.30.2", "type": "open", "file": "/tmp/gdsfactory/Unnamed_050aa4c5.gds"}


In [24]:
metal_contacts.add_label("E", position=(0,0),layer=pdk.get_glayer(metal_layer_name))

directions=((1,0),(0,1),(-1,0),(0,-1))
for direction in directions:
    metal_contacts.add_label("B", position=tuple(np.asarray(direction)*get_mid_ring(bdims,edims)/2),
                        layer=pdk.get_glayer(metal_layer_name))
    metal_contacts.add_label("C", position=tuple(np.asarray(direction)*get_mid_ring(cdims,bdims)/2),
                        layer=pdk.get_glayer(metal_layer_name))


In [25]:
metal_contacts.show()

2025-09-02 07:24:41.383 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.3", "klayout_version": "0.30.2", "type": "reload", "file": "/tmp/gdsfactory/Unnamed_050aa4c5.gds"}


In [26]:
top_level.add_ref(metal_contacts)

ComponentReference (parent Component "Unnamed_050aa4c5", ports [], origin (0.0, 0.0), rotation 0.0, x_reflection False)

In [27]:
top_level.show()

/headless/conda-env/miniconda3/envs/GLdev/lib/python3.10/site-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 2 in 'BJT'
  gdspath = component.write_gds(
2025-09-02 07:24:41.890 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.3", "klayout_version": "0.30.2", "type": "reload", "file": "/tmp/gdsfactory/BJT.gds"}


In [28]:
drc_bjt= rectangle(size=tuple(np.asarray(bjt_dims[2])),
                  layer=(127,5),centered=True)
lvs_bjt= rectangle(size=tuple(np.asarray(bjt_dims[0])-2*comp_min_sep),
                  layer=(118,5),centered=True)

In [29]:
drc_bjt.show()

2025-09-02 07:24:42.449 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.3", "klayout_version": "0.30.2", "type": "open", "file": "/tmp/gdsfactory/rectangle_f36c0b6e.gds"}


In [30]:
lvs_bjt.show()

2025-09-02 07:24:42.769 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.3", "klayout_version": "0.30.2", "type": "open", "file": "/tmp/gdsfactory/rectangle_f41950f2.gds"}


In [31]:
top_level.add_ref(drc_bjt)
top_level.add_ref(lvs_bjt)

ComponentReference (parent Component "rectangle_f41950f2", ports ['e1', 'e2', 'e3', 'e4'], origin (0.0, 0.0), rotation 0.0, x_reflection False)

In [32]:
top_level.show()

2025-09-02 07:24:43.171 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.3", "klayout_version": "0.30.2", "type": "reload", "file": "/tmp/gdsfactory/BJT.gds"}


In [33]:
top_level.pprint_ports()

In [34]:
bjt=top_level.flatten()

In [35]:
bjt.show()

/headless/conda-env/miniconda3/envs/GLdev/lib/python3.10/site-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 1 in 'Unnamed_8efdf4d8'
  gdspath = component.write_gds(
2025-09-02 07:24:44.029 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.3", "klayout_version": "0.30.2", "type": "open", "file": "/tmp/gdsfactory/Unnamed_8efdf4d8.gds"}


In [36]:
bjt.pprint_ports()

In [37]:
from pathlib import Path
import os
import subprocess
import tempfile
magicrc_file = Path(os.environ['PDKPATH']) / "libs.tech" / "magic" / f"{os.environ['PDK']}.magicrc"
magicrc_file

PosixPath('/foss/pdks/gf180mcuD/libs.tech/magic/gf180mcuD.magicrc')

In [38]:
def extract_pex(design, path_to_dir):
    design_name=design.name
    if not path_to_dir.exists():
        path_to_dir.mkdir(parents=True, exist_ok=False)
    
    pex_path = path_to_dir / f"{design_name}_pex.spice"
    gds_path = path_to_dir / f"{design_name}.gds"
    
    design.write_gds(str(gds_path))
        
    magic_script_content = f"""
    drc off            
    gds flatglob *\\$\\$*
    gds read {gds_path}
    
    flatten {design_name}
    load {design_name}
    select top cell
    extract do local
    extract all
    ext2sim labels on
    ext2sim
    extresist tolerance 10
    extresist
    ext2spice lvs
    ext2spice cthresh 0
    ext2spice extresist on
    ext2spice -o {str(pex_path)}
    exit
    """
    
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as magic_script_file:
        magic_script_file.write(magic_script_content)
        magic_script_path = magic_script_file.name
        
    magic_cmd = f"bash -c 'magic -rcfile {magicrc_file} -noconsole -dnull < {magic_script_path}'",
    magic_subproc = subprocess.run(
        magic_cmd, 
        shell=True,
        check=True,
        capture_output=True
    )
    
    magic_subproc_code = magic_subproc.returncode
    magic_subproc_out = magic_subproc.stdout.decode('utf-8')
    print(magic_subproc_out)

In [39]:
path_to_dir = Path("/foss/designs/gLayout/tutorial").resolve() / "ext" / bjt.name
extract_pex(bjt, path_to_dir)

/tmp/ipykernel_36686/1276761136.py:9: UserWarning: Unnamed cells, 1 in 'Unnamed_8efdf4d8'
  design.write_gds(str(gds_path))
2025-09-02 07:24:45.114 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/foss/designs/gLayout/tutorial/ext/Unnamed_8efdf4d8/Unnamed_8efdf4d8.gds'



Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Using technology "gf180mcuD", version 1.0.525-0-gf2e289d
Library written using GDS-II Release 6.0
Library name: library
Reading "Unnamed_8efdf4d8".
Moving label "E" from metal1 to pdiffc in cell Unnamed_8efdf4d8.
Moving label "B" from metal1 to nsubdiffcont in cell U